In [1]:
#basic imports
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import pymongo as pm
import numpy as np
import h5py
import scipy as sp
import scipy.stats as stats
import tables
import os

In [2]:
file_path="/share/kalanit/biac2/kgs/projects/Dawn/CS431/data/"

#read in the mat files
fmri_dataset=h5py.File(os.path.join(file_path,"estimated_responses.mat"),'r')

In [3]:
list(fmri_dataset.items())

[(u'dataTrnS1', <HDF5 dataset "dataTrnS1": shape (1750, 25915), type "<f8">),
 (u'dataTrnS2', <HDF5 dataset "dataTrnS2": shape (1750, 26329), type "<f8">),
 (u'dataValS1', <HDF5 dataset "dataValS1": shape (120, 25915), type "<f8">),
 (u'dataValS2', <HDF5 dataset "dataValS2": shape (120, 26329), type "<f8">),
 (u'roiS1', <HDF5 dataset "roiS1": shape (1, 25915), type "<f8">),
 (u'roiS2', <HDF5 dataset "roiS2": shape (1, 26329), type "<f8">),
 (u'voxIdxS1', <HDF5 dataset "voxIdxS1": shape (1, 25915), type "<f8">),
 (u'voxIdxS2', <HDF5 dataset "voxIdxS2": shape (1, 26329), type "<f8">)]

In [4]:
fmri_dataset.keys()

[u'dataTrnS1',
 u'dataTrnS2',
 u'dataValS1',
 u'dataValS2',
 u'roiS1',
 u'roiS2',
 u'voxIdxS1',
 u'voxIdxS2']

In [ ]:
#f = tables.open_file("data/EstimatedResponses.mat")

In [5]:
fmri_dataset.close()